In [6]:
import requests
import psycopg2
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
from src.databases import Postgresql

In [7]:
#lista de Vinhos
lista = []
lista_vinhos = []

for i in range(1,84):

    r = requests.get(f'https://www.wine.com.br/browse.ep?cID=100851&exibirEsgotados=false&pn={i}&listagem=horizontal&sorter=featuredProducts-desc&filters=cVINHOS')
    soup = BeautifulSoup(r.content,'html.parser')
    container = soup.find_all('div',{'class':'ProductDisplay-name'})
    for col in container: # contém todos os vinhos
        for art in col.find_all('a',href=True): #vinhos em cada coluna
            lista = 'https://www.wine.com.br{}'.format(art['href'])
            lista_vinhos.append(lista)
            

#Sku
lst_sku=[]
for i in range(1,84):
    PAGE = f'https://www.wine.com.br/vinhos/cVINHOS-p{i}.html'
    if i % 10 == 0:
        print(f'Raspando pagina {i}')

    resultado = requests.get(PAGE)
    bs = BeautifulSoup(resultado.content, 'html.parser')
    html_data = bs.find_all('div', {'class': 'ProductDisplay-name'})
    sku = np.array([elem.a['data-product-sku'] for elem in html_data])
    
    for elem in html_data:
        lst_sku.append(elem.a['data-product-sku'])

Raspando pagina 10
Raspando pagina 20
Raspando pagina 30
Raspando pagina 40
Raspando pagina 50
Raspando pagina 60
Raspando pagina 70
Raspando pagina 80


In [8]:
#capturando os comentarios do sommelier
comments_sommelier = []
for i, j in enumerate(lista_vinhos):
    
    tag = j
    resultado = requests.get(tag)
    bs = BeautifulSoup(resultado.content, 'html.parser')
    html_data = bs.find_all('div', {'class': 'ReadMore-text'}, limit=1)

    for elem in html_data:
        comments_sommelier.append({'sku':lst_sku[i], 'comments':elem.string})

In [9]:
#Gerando os dataframes
dataset_comments = pd.DataFrame(comments_sommelier)
dataset_comments.set_index('sku', inplace= True)
dataset_comments = dataset_comments.reset_index()

In [13]:
query = """ CREATE TABLE comments_sommelier ( sku TEXT,
                                              comments VARCHAR(1000))  """



bd = Postgresql()
bd.create_table(query=query, user='postgres' , password='bruno22#' , host= 'localhost', port= '5432', database = 'brunods')
bd.insert_data(df = dataset_comments, tabela = ' comments_sommelier',user='postgres' , password='bruno22#' , host= 'localhost', port= '5432', database = 'brunods')

Tabela criada no postgresql com sucesso 
Conexão com Postgresql fechada


In [14]:
dataset_comments.head()

,sku,comments
0,18479,"\nRosé francês frutado, leve, fresco e com doç..."
1,3170,"\nCom notas de frutas brancas, mel, pão e ferm..."
2,22807,\n Pinot Noir da Borgonha com aromas de frutas...
3,22698,"\nUm dos mais nobres champagnes do mundo, o Mo..."
4,22696,\nO primeiro champagne para ser apreciado com ...
